
# 🧠 Attention From Scratch: A 5‑Lesson Math‑First Mini‑Course (Python Only)

**Audience:** Freshman → Senior undergrads in Math/CS (or motivated self‑learners)  
**Goal:** Understand the math *under the hood* of modern AI (especially Transformers) by coding everything from first principles—no NumPy, no PyTorch.  
**Format:** 5 lessons, each with *explanations, code, exercises,* and *print‑outs* so you see what's happening.

### What you'll build
1. **Linear Algebra Toolkit** (vectors, dot products, matrix multiply, transpose, cosine similarity)
2. **Probability Toolkit** (softmax, cross‑entropy, batch operations)
3. **Optimization Basics** (derivatives, numerical gradient, gradient descent)
4. **Tiny Neural Net From Scratch** (solve XOR with manual backprop)
5. **Scaled Dot‑Product Attention** (the core of Transformers), including **causal masking**

> **Why no heavy libraries?** To remove the “black box” and make the math tangible. When you *write* `matmul`, `softmax`, or `attention` yourself, you *feel* why these operations matter.



## 🏁 Getting started

- Run each cell in order (Shift+Enter).
- Read the printed output. Tweak values and re‑run to build intuition.
- Each lesson ends with **Try it** prompts so you can stretch further.


In [ ]:

# Minimal dependencies: only Python stdlib
import math, random
random.seed(42)

def banner(title):
    print("\n" + "="*80)
    print(title)
    print("="*80)



## 🔧 Core utilities we’ll reuse
We keep these helpers tiny and explicit. If you don't understand a function, scroll back and study it—there's no magic here.


In [ ]:

def zeros_like(A):
    '''Return a zero-structure with the same shape as A (list or list of lists).'''
    if isinstance(A, list) and len(A) > 0 and isinstance(A[0], list):
        return [[0.0 for _ in row] for row in A]
    return [0.0 for _ in A]

def mat_shape(A):
    '''Return shape of a vector or matrix stored as nested Python lists.'''
    if isinstance(A[0], list): return (len(A), len(A[0]))
    return (len(A), )

def matmul(A, B):
    '''
    Matrix multiply A (m x n) by B (n x p) -> (m x p).
    Implemented explicitly to show the triple loop.
    '''
    m, n = len(A), len(A[0])
    assert len(B) == n, f"shape mismatch {mat_shape(A)} @ {mat_shape(B)}"
    p = len(B[0])
    out = [[0.0]*p for _ in range(m)]
    for i in range(m):
        for k in range(n):
            aik = A[i][k]
            for j in range(p):
                out[i][j] += aik * B[k][j]
    return out

def transpose(A):
    '''Return the transpose of matrix A (list of rows -> list of columns).'''
    return [list(row) for row in zip(*A)]

def add(A, B):
    '''Elementwise add for vectors or matrices (nested lists).'''
    if isinstance(A[0], list):
        return [[a+b for a,b in zip(ra, rb)] for ra, rb in zip(A,B)]
    return [a+b for a,b in zip(A,B)]

def sub(A, B):
    '''Elementwise subtract for vectors or matrices (nested lists).'''
    if isinstance(A[0], list):
        return [[a-b for a,b in zip(ra, rb)] for ra, rb in zip(A,B)]
    return [a-b for a,b in zip(A,B)]

def scalar_mul(A, s):
    '''Multiply every element of vector or matrix A by scalar s.'''
    if isinstance(A[0], list):
        return [[a*s for a in row] for row in A]
    return [a*s for a in A]

def dot(u, v):
    '''Dot product of two vectors.'''
    return sum(ui*vi for ui,vi in zip(u,v))

def norm(v):
    '''Euclidean (L2) norm of a vector, with tiny floor for stability.'''
    return math.sqrt(max(1e-12, dot(v,v)))

def cosine_similarity(u, v):
    '''Cosine similarity of two vectors (angle-based similarity).'''
    return dot(u, v) / (norm(u) * norm(v))

def argmax(xs):
    '''Return the index of the largest value in a list.'''
    return max(range(len(xs)), key=lambda i: xs[i])

def one_hot(idx, n):
    '''One-hot vector of length n with 1 at idx.'''
    v = [0.0]*n
    v[idx] = 1.0
    return v

def softmax(xs):
    '''
    Convert a list of real numbers to a probability distribution.
    We subtract max(xs) for numerical stability (avoids overflow).
    '''
    m = max(xs)
    exps = [math.exp(x - m) for x in xs]
    Z = sum(exps)
    return [e / Z for e in exps]

def softmax_rows(M):
    '''Row-wise softmax for a 2D list (matrix).'''
    return [softmax(row) for row in M]

def cross_entropy(p, q):
    '''Cross-entropy between two probability vectors p and q.'''
    eps = 1e-12
    return -sum(pi * math.log(max(qi, eps)) for pi, qi in zip(p,q))

def cross_entropy_rows(P, Q):
    '''Average cross-entropy across batches (row-wise).'''
    return sum(cross_entropy(p, q) for p, q in zip(P, Q)) / len(P)

def gelu(x):
    '''GELU activation (approximation).'''
    return 0.5 * x * (1.0 + math.tanh(math.sqrt(2.0/math.pi)*(x + 0.044715*(x**3))))

def relu(x):
    '''ReLU activation.'''
    return x if x > 0 else 0.0

def relu_vec(v):
    '''Apply ReLU to every element of a vector.'''
    return [relu(x) for x in v]

def apply_rowwise(A, fn):
    '''Apply a scalar function elementwise to each row of a matrix.'''
    return [[fn(x) for x in row] for row in A]

def randn_matrix(m, n, std=0.02):
    '''Gaussian random matrix with mean 0 and standard deviation std.'''
    return [[random.gauss(0.0, std) for _ in range(n)] for _ in range(m)]



---
# 🧩 LESSON 1 — Linear Algebra Foundations

**Concepts:** vectors, dot products, norms, angles, matrix multiplication, transpose, cosine similarity.  
**Why it matters:** Every neural net layer is basically `y = xW + b`. Attention uses `Q @ K^T`.  
Understanding these basics makes the rest feel natural.


In [ ]:

banner("LESSON 1: Linear Algebra Foundations")

# Example vectors
u = [1, 2, 3]
v = [2, 1, 0]

print("u =", u)
print("v =", v)
print("dot(u, v) =", dot(u, v))
print("norm(u)   =", round(norm(u), 4))
print("cosine_similarity(u, v) =", round(cosine_similarity(u, v), 4))

# Example matrices
A = [
    [1, 2],
    [3, 4],
    [5, 6],
]  # shape (3x2)

B = [
    [7, 8, 9],
    [10, 11, 12],
]  # shape (2x3)

print("\nMatrix A (3x2):", A)
print("Matrix B (2x3):", B)

C = matmul(A, B)  # (3x3)
print("\nC = A @ B (3x3):")
for row in C:
    print([round(x, 3) for x in row])

Ct = transpose(C)
print("\ntranspose(C):")
for row in Ct:
    print([round(x, 3) for x in row])

print("\nTry it:")
print("1) Change entries of u and v and observe cosine similarity.")
print("2) Create your own matrices and check shape compatibility for matmul.")



---
# 🎲 LESSON 2 — Probability & Softmax

**Concepts:** turning scores (logits) into probabilities (softmax), comparing distributions (cross‑entropy).  
**Why it matters:** Classifiers output logits; training minimizes cross‑entropy with the target distribution.


In [ ]:

banner("LESSON 2: Probability & Softmax")

logits = [2.0, 1.0, 0.1]
probs = softmax(logits)
target = one_hot(0, 3)  # pretend the correct class is index 0

print("logits:", logits)
print("softmax(logits):", [round(p, 4) for p in probs])
print("target one-hot:", target)
print("cross_entropy(target, probs):", round(cross_entropy(target, probs), 6))

# Row-wise demo (mini-batch of two examples)
batch_logits = [
    [1.0, -1.0, 0.0],
    [0.1, 0.2, 0.3],
]
batch_probs = softmax_rows(batch_logits)
batch_targets = [one_hot(2,3), one_hot(1,3)]

print("\nBatch softmax:")
for row in batch_probs:
    print([round(x, 4) for x in row])

print("Batch CE:", round(cross_entropy_rows(batch_targets, batch_probs), 6))

print("\nTry it:")
print("1) Make the correct class logit larger and see CE go down.")
print("2) Add a huge number to all logits—does softmax change? Why not? (Hint: invariance)")



---
# 📉 LESSON 3 — Calculus & Optimization (Backprop Basics)

**Concepts:** gradient, numerical vs. analytical derivatives, gradient descent.  
**Why it matters:** Training = minimizing a loss by following the gradient downhill.


In [ ]:

banner("LESSON 3: Calculus & Optimization")

def gradient_descent_1d(f, df, x0, lr=0.1, steps=25, verbose=True):
    x = x0
    history = []
    for t in range(steps):
        grad = df(x)
        x = x - lr * grad
        fx = f(x)
        history.append((t+1, x, fx))
        if verbose:
            print(f"step {t+1:02d}: x={x:.5f}, f(x)={fx:.8f}, grad={grad:.5f}")
    return x, history

def numeric_grad(f, x, eps=1e-5):
    return (f(x+eps) - f(x-eps)) / (2*eps)

# Minimize f(x) = (x - 3)^2
f  = lambda x: (x-3)**2
df = lambda x: 2*(x-3)

x_star, hist = gradient_descent_1d(f, df, x0=0.0, lr=0.2, steps=15, verbose=True)
print("\nargmin x* ≈", round(x_star, 6), "f(x*) =", round(f(x_star), 12))
print("numeric_grad at x=3.0 ->", numeric_grad(f, 3.0))

print("\nTry it:")
print("1) Change lr (learning rate) to 1.5 and watch it diverge.")
print("2) Change steps to 50 and watch convergence slow/fast.")
print("3) Replace f with a non-convex function (e.g., sin(x) + 0.1x^2).")



---
# 🧪 LESSON 4 — Tiny Neural Net From Scratch (XOR)

**Concepts:** linear layers, activations, softmax + cross‑entropy, *manual* backprop.  
**Why it matters:** Before Transformers, all deep nets are compositions of linear maps and nonlinearities.


In [ ]:

banner("LESSON 4: Tiny NN From Scratch (XOR)")

def init_layer(in_dim, out_dim, std=0.5):
    W = randn_matrix(out_dim, in_dim, std=std)  # rows=out_dim
    b = [[0.0] for _ in range(out_dim)]
    return W, b

def lin_forward(X, W, b):
    # X: (N x D), W: (O x D), b: (O x 1) -> (N x O)
    return add(matmul(X, transpose(W)), transpose(b))

def relu_forward(X):
    return apply_rowwise(X, relu)

def softmax_forward(X):
    return softmax_rows(X)

def xor_dataset():
    X = [
        [0.0, 0.0],
        [0.0, 1.0],
        [1.0, 0.0],
        [1.0, 1.0],
    ]
    Y = [
        [1.0, 0.0],  # class 0
        [0.0, 1.0],  # class 1
        [0.0, 1.0],
        [1.0, 0.0],
    ]
    return X, Y

def train_xor(epochs=2000, lr=0.1, hidden=4, verbose=True):
    X, T = xor_dataset()      # X: (4x2), T: (4x2 one-hot)
    D_in, D_h, D_out = 2, hidden, 2

    # Init layers
    W1, b1 = init_layer(D_in, D_h, std=0.8)  # (h x 2), (h x 1)
    W2, b2 = init_layer(D_h, D_out, std=0.8) # (2 x h), (2 x 1)

    for epoch in range(epochs):
        # Forward
        Z1 = lin_forward(X, W1, b1)        # (4 x h)
        A1 = relu_forward(Z1)
        Z2 = lin_forward(A1, W2, b2)       # (4 x 2)
        Y  = softmax_forward(Z2)           # (4 x 2)

        # Loss (Cross-Entropy)
        loss = cross_entropy_rows(T, Y)

        # Backprop (manual, small network)
        # dL/dZ2 = Y - T   (from softmax + CE)
        dZ2 = [[(y - t) for y,t in zip(yrow, trow)] for yrow, trow in zip(Y, T)]  # (4 x 2)

        # Grad W2 = A1^T @ dZ2
        dW2 = matmul(transpose(A1), dZ2)   # (h x 2)
        db2 = [[sum(col)] for col in transpose(dZ2)]  # (2 x 1)

        # Back to layer1: dA1 = dZ2 @ W2
        dA1 = matmul(dZ2, W2)              # (4 x h)
        # ReLU backprop: dZ1 = dA1 * (Z1 > 0)
        dZ1 = []
        for i in range(len(Z1)):
            dZ1.append([dA1[i][j] * (1.0 if Z1[i][j] > 0 else 0.0) for j in range(len(Z1[0]))])

        # Grad W1 = X^T @ dZ1 (then transpose to match shape)
        dW1 = transpose(matmul(transpose(X), dZ1))    # (h x 2)
        db1 = [[sum(col)] for col in transpose(dZ1)]  # (h x 1)

        # Gradient step
        W2 = sub(W2, scalar_mul(dW2, lr))
        b2 = sub(b2, scalar_mul(db2, lr))
        W1 = sub(W1, scalar_mul(dW1, lr))
        b1 = sub(b1, scalar_mul(db1, lr))

        if verbose and (epoch+1) % 500 == 0:
            preds = [argmax(y) for y in Y]
            acc = sum(int(p == argmax(t)) for p,t in zip(preds, T)) / len(T)
            print(f"epoch {epoch+1:4d} | loss {loss:.4f} | acc {acc:.2f}")

    # Final sanity check
    print("\nPredictions after training: (probabilities then class)")
    X, T = xor_dataset()
    # Run one more forward for printing
    Z1 = lin_forward(X, W1, b1); A1 = relu_forward(Z1)
    Z2 = lin_forward(A1, W2, b2); Y = softmax_forward(Z2)
    for x, y, t in zip(X, Y, T):
        print(f"X={x} -> P={ [round(v,3) for v in y] } -> pred={argmax(y)} | true={argmax(t)}")

    return (W1, b1, W2, b2)

# Train the tiny net
_ = train_xor(epochs=2000, lr=0.1, hidden=4, verbose=True)

print("\nTry it:")
print("1) Change hidden size to 2 or 8. Does it learn faster/slower?")
print("2) Replace ReLU with GELU and compare.")
print("3) Try MSE loss instead of cross-entropy and observe differences.")



---
# 🔭 LESSON 5 — Scaled Dot‑Product Attention

**Concepts:** projections (Q, K, V), similarity via dot products, scaling by √d, softmax over scores, causal masks.  
**Why it matters:** This operation is the *heart* of GPT‑style Transformers.


In [ ]:

banner("LESSON 5: Scaled Dot-Product Attention")

def causal_mask(n):
    '''Lower-triangular mask (n x n) with 0 above diagonal, 1 elsewhere.'''
    return [[1.0 if j <= i else 0.0 for j in range(n)] for i in range(n)]

def masked_fill(M, mask, fill_value=-1e9):
    '''Like PyTorch masked_fill, for nested lists.'''
    out = []
    for i in range(len(M)):
        row = []
        for j in range(len(M[0])):
            row.append(M[i][j] if mask[i][j] > 0.5 else fill_value)
        out.append(row)
    return out

def attention(Q, K, V, causal=False):
    '''
    Q: (T x d), K: (T x d), V: (T x dv)
    returns: (T x dv), (T x T) weights, (T x T) scores
    '''
    d = len(Q[0])
    scores = matmul(Q, transpose(K))               # (T x T)
    scores = [[s / math.sqrt(d) for s in row] for row in scores]
    if causal:
        M = causal_mask(len(scores))
        scores = masked_fill(scores, M, fill_value=-1e9)
    weights = softmax_rows(scores)                 # (T x T)
    out = matmul(weights, V)                       # (T x dv)
    return out, weights, scores

def linear_project(X, W):
    # X: (T x d_in), W: (d_in x d_out)  -> (T x d_out)
    return matmul(X, W)

def demo_attention(T=4, d=6, dv=6, causal=True):
    # Toy token embeddings (sequence length T, dim d)
    X = [[random.gauss(0.0, 1.0) for _ in range(d)] for _ in range(T)]

    # Learnable projections (random init for demo)
    Wq = randn_matrix(d, d, std=0.4)
    Wk = randn_matrix(d, d, std=0.4)
    Wv = randn_matrix(d, dv, std=0.4)

    Q = linear_project(X, Wq)   # (T x d)
    K = linear_project(X, Wk)   # (T x d)
    V = linear_project(X, Wv)   # (T x dv)

    Y, W, S = attention(Q, K, V, causal=causal)

    print(f"Sequence length T={T}, dim d={d}, value dim dv={dv}, causal={causal}")
    print("\nFirst token embedding X[0]:")
    print([round(x,3) for x in X[0]])

    print("\nScaled scores S = Q @ K^T / sqrt(d):")
    for row in S:
        print([round(s, 3) for s in row])

    print("\nAttention weights (rows sum to 1):")
    for row in W:
        print([round(w, 3) for w in row])

    print("\nOutput (context) vectors Y (one per token):")
    for row in Y:
        print([round(y, 3) for y in row])

demo_attention(T=4, d=6, dv=6, causal=True)

print("\nTry it:")
print("1) Set causal=False and observe attention to 'future' tokens.")
print("2) Increase d to 32 and note how scaling affects softmax sharpness.")
print("3) Make two tokens identical in X and watch their mutual attention grow.")
